In [1]:
from transformers import pipeline, AutoFeatureExtractor
import datasets
import pandas as pd
import numpy as np
import glob

/home/dkemper/anaconda3/envs/audio-similarity-search/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Read dataset

In [2]:
# Load locally from disk
ds = datasets.load_dataset('esc50')

ds = ds.cast_column("audio", datasets.Audio(sampling_rate=16000))

In [3]:
MODEL_NAME = 'facebook/wav2vec2-large'

feature_extractor = AutoFeatureExtractor.from_pretrained(
    MODEL_NAME, do_normalize=True, return_attention_mask=True
)

In [4]:
MAX_DURATION = 5.0

def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * MAX_DURATION),
        truncation=True,
        return_attention_mask=True,
    )
    return inputs

In [5]:
ds_encoded = ds.map(
    preprocess_function,
    remove_columns=["audio"],
    batched=True,
    batch_size=100,
    num_proc=1,
)

In [6]:
id2label_fn = ds["train"].features['label'].int2str
id2label = {
    str(i): id2label_fn(i)
    for i in range(len(ds_encoded["train"].features["label"].names))
}
label2id = {v: k for k, v in id2label.items()}

### Finetune model

In [7]:
from transformers import AutoModelForAudioClassification

num_labels = len(id2label)

model = AutoModelForAudioClassification.from_pretrained(
    MODEL_NAME,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
from transformers import TrainingArguments

model_name = MODEL_NAME.split("/")[-1]
batch_size = 8
gradient_accumulation_steps = 1
num_train_epochs = 10

training_args = TrainingArguments(
    f"{model_name}-finetuned-gtzan",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    push_to_hub=False,
)

In [9]:
import evaluate
import numpy as np

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [10]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=ds_encoded["train"],
    eval_dataset=ds_encoded["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

trainer.save_model('audio_classification_model')

/tmp/ipykernel_48668/2240248666.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,3.441200,3.391680,0.096000
2,3.134100,3.381449,0.116000
3,2.578500,2.433555,0.344000
4,1.944000,2.083581,0.420000
5,1.327700,1.699644,0.492000
6,1.452600,1.370617,0.640000
7,1.267400,1.327884,0.656000
8,0.598100,1.114468,0.744000
9,0.468600,0.981883,0.764000
10,0.405100,0.905049,0.780000


### Inference

In [11]:
pipe = pipeline('audio-classification', model='audio_classification_model')

# Classify the audio file at the given path
def classify_audio(filepath):
    preds = pipe(filepath)
    outputs = {}
    outputs[preds[0]["label"]] = preds[0]["score"]
    return next(iter(outputs))

val_files = glob.glob('esc50/val/**/*.wav', recursive=True)
scores_list = []
for file in val_files:
    target_category = file.split('/')[2]
    inferred_category = classify_audio(file)

    scores_list.append(1) if target_category == inferred_category else scores_list.append(0)

print('Accuracy: {}'.format(scores_list.count(1) / len(scores_list)))

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Accuracy: 0.76


### Some classification examples:

In [12]:
classify_audio('esc50/val/airplane/5-251971-A-47.wav')

'sea_waves'

In [13]:
classify_audio('esc50/val/breathing/5-261464-A-23.wav')

'cow'

In [14]:
classify_audio('esc50/val/can_opening/5-260011-A-34.wav')

'can_opening'

In [15]:
classify_audio('esc50/val/door_wood_knock/5-256512-A-30.wav')

'door_wood_knock'

In [16]:
classify_audio('esc50/val/vacuum_cleaner/5-263902-A-36.wav')

'vacuum_cleaner'

In [17]:
classify_audio('esc50/val/insects/5-233787-A-7.wav')

'insects'

In [18]:
classify_audio('esc50/val/train/5-199284-B-45.wav')

'train'

In [19]:
classify_audio('esc50/val/rooster/5-234879-B-1.wav')

'rooster'